In [13]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np 

In [14]:
def read(his_file,sim_num): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 

# Remember all arrays are (time,kmax,jmax,imax) contrary to Matlab     
    imax=len(x[1,:])
    jmax=len(y[:,1])
     
# read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]

#Get Stretching function 
    N=60 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 

# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,len(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1

# Find water level for all locations at last tidal cycle  
    
# Calculate maximum water level change in a tidal cycle before and after the patch
# Probe the region infront of the patch by adding 5 points in vertical 
# Probe the region behind the patch by adding 5 points in vertical 
# Water level change in the middle transect before and after the patch

    grad_diff=0.0     
    for t in range (t_tidal_srt,t_tidal_end+1): 
                # gradient global in y direction 
        grad=abs(eta[t,jmin_plant-3,imid]-eta[t,jmax_plant+3,imid]) \
                  /abs(y[jmin_plant-3,imid]-y[jmax_plant+3,imid])
        if (grad>grad_diff):
            jstore=jmin_plant-3
            istore=imid 
        grad_diff=max(grad_diff,grad)   
        
        # gradient global in x direction 
        grad=abs(eta[t,jmid,imin_plant-3]-eta[t,jmid,imax_plant+3]) \
                /abs(x[jmid,imin_plant-3]-x[jmid,imax_plant+3])
        if (grad>grad_diff):
            jstore=jmid
            istore=imin_plant-3
        grad_diff=max(grad_diff,grad)
   
        # Probe gradient locally in x and y direction 
        for j in range(jmin_plant-5,jmax_plant+6):
            for i in range(imin_plant-5,imax_plant+6): 
        
                grad_diff=abs(eta[t,j,i-1]-eta[t,j,i])/abs(x[j,i-1]-x[j,i])
                if (grad>grad_diff):
                    jstore=j
                    istore=i  
                grad_diff=max(grad_diff,grad)
                
                grad=abs(eta[t,j-1,i]-eta[t,j,i])/abs(y[j-1,i]-y[j,i])                
                if (grad>grad_diff):
                    jstore=j
                    istore=i 
                grad_diff=max(grad_diff,grad)
                    
    print "Maximum water level, istore, jstore"
    print grad_diff, istore, jstore 
    grad_ret=[grad_diff,istore,jstore] 
    return grad_ret
#    f.write("%s\t, %s\t %s\t %s\t \n" % (sim_num, grad_wl, istore, jstore))

In [17]:
 '''Output from vegetation module COAWST 
Mean gradient of water level around vegetation p)[mts.]
'''    

total_sim_list=16

# Declare some arrays to be filled later 
grad_diff=[None]*total_sim_list 
pcnt_min_grad_diff=[None]*total_sim_list
istore=[None]*total_sim_list 
jstore=[None]*total_sim_list

for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/'
    path2='veg_test_his_'+ str(sim_num)+'_ext' + '.nc'
    url=path1+path2
    print url

    grad_returns=read(url,sim_num) 
    
    grad_diff[sim_num]=grad_returns[0]
    istore[sim_num]=grad_returns[1]
    jstore[sim_num]=grad_returns[2]

min_grad_diff=min(grad_diff[1:total_sim_list])

fname='Grad_Water_level_output.txt'
f = open(fname, 'w')
f.write("Case_no, Grad_Water_level(mts.), pcnt_min_grad_ret, istore, jstore: \n")

for sim_num in range(1,total_sim_list):
    pcnt_min_grad_diff[sim_num]=(grad_diff[sim_num]-min_grad_diff)*100.00/min_grad_diff 
    
    f.write("%s\t, %s\t, %s\t, %d\t, %d\t \n" % (sim_num, grad_diff[sim_num], \
               pcnt_min_grad_diff[sim_num], istore[sim_num], jstore[sim_num] ))
                   
f.close()



/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_1_ext.nc
Last tidal cycle starts and ends at this instance
180 252
Maximum water level, istore, jstore
1.14128459245e-06 55 59
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_2_ext.nc
Last tidal cycle starts and ends at this instance
180 252
Maximum water level, istore, jstore
1.06789171696e-06 55 59
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_3_ext.nc
Last tidal cycle starts and ends at this instance
180 252
Maximum water level, istore, jstore
1.11872330308e-06 55 59
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_4_ext.nc
Last tidal cycle starts and ends at this instance
180 252
Maximum water level, istore, jstore
1.12950801849e-06 55 59
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_5_ext.nc
Last tidal cycle starts and ends at this instance
180 252
Maximum water level, istore, jstore
1.13985966891e-06 55 59
/media/gadar/DATADRI